# Generating the data and push it to the SQL database

### This notebook is used to generate the random data and push it to the postgres so that airflow workflow can fetch it, convert it to json and push to the mongo DB

In [1]:
try:
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2
except:
    !pip install pandas numpy faker sqlalchemy psycopg2-binary pymongo
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2

  Using cached pandas-1.4.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached Faker-13.12.0-py3-none-any.whl (1.6 MB)
  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached pymongo-4.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (471 kB)


#### Generating the csv file

In [2]:
from faker import Faker
import csv
output=open('data.csv','w')
fake=Faker()
header=['name','age','street','city','state','zip','lng','lat']
mywriter=csv.writer(output)
mywriter.writerow(header)
for r in range(1000):
    row =[fake.name(),fake.random_int(min=18,max=80, step=1), 
                       fake.street_address(), fake.city(),fake.state(),
                       fake.zipcode(),fake.longitude(),fake.latitude()]
    #print(row)
    mywriter.writerow(row)

output.close()

#### Create the connection to the database
Due to a bug in the libpq packages for Mac M1 laptops, a workaround is to run the containers with rosetta or on intel based Linux machine ref link: https://stackoverflow.com/questions/62807717/how-can-i-solve-postgresql-scram-authentifcation-problem

In [5]:
host='ibrahim_postgres'
username='postgres'
password='password123'
database='deproject'
port='5432'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
print(engine.table_names())

['data']


/tmp/ipykernel_75/1324543537.py:8: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


#### Send the csv data to the postgres database

In [7]:
df = pd.read_csv('./data.csv')
df.head()

,name,age,street,city,state,zip,lng,lat
0,Dr. Rachel Cohen,79,9248 Andrew Viaduct Apt. 249,New Tylerstad,New Mexico,81744,46.529693,57.701027
1,Jocelyn Abbott,80,339 Morrow Highway Suite 359,Estesborough,West Virginia,32890,-12.151620,-19.978748
2,Erica Love,67,069 Marc Rapid Apt. 021,North John,Wyoming,93096,9.373608,-62.521178
3,Alexis Lee,29,97666 Robinson Port Apt. 566,Bradyfort,New Jersey,94657,178.078593,-5.186961
4,Theresa Butler,65,146 Natasha Shoal,Stephanieville,Florida,93566,-6.136323,68.989732


In [8]:
df.to_sql('data',engine, if_exists='replace', index=False)

1000